# Restaurants in the 'Coolest' Neighborhoods in Miami

In [270]:

from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('expand_frame_repr', False)

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import requests # library to handle requests

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

In [271]:
content = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami').text
parser = BeautifulSoup(content, "lxml")

In [272]:
#print(parser.prettify())

### Get rows from table

In [273]:
table = parser.find(lambda tag: tag.name=='table')

rowList = []
 
rows = table.select('tr')

for tr in rows:
    td = tr.findAll(lambda tag: tag.name=='td')
    if(len(td) > 0):
        row = [tr.text.replace('\n', ' ').strip() for tr in td]
    else:
        th = rows[0].findAll(lambda tag: tag.name=='th')
        row = [tr.text.replace('\n', ' ').strip() for tr in th]
       
    rowList.append(row)

In [141]:
#print(rowList)

### Convert to dataframe and cleanup

In [274]:
df = pd.DataFrame(rowList)
selected_columns = df[[0,5]]
df = selected_columns.copy()
df.columns = ['Neighborhood', 'Coordinates']
df = df.drop(df.index[0])
print (df.to_string(index = False))

                  Neighborhood     Coordinates
                    Allapattah  25.815,-80.224
 Arts & Entertainment District  25.799,-80.190
                      Brickell  25.758,-80.193
                   Buena Vista  25.813,-80.192
                 Coconut Grove  25.712,-80.257
                     Coral Way  25.750,-80.283
               Design District  25.813,-80.193
                      Downtown  25.774,-80.193
                     Edgewater  25.802,-80.190
                       Flagami  25.762,-80.316
             Grapeland Heights  25.792,-80.258
               Health District                
                  Liberty City  25.832,-80.225
                  Little Haiti  25.824,-80.191
                 Little Havana  25.773,-80.215
                   Lummus Park  25.777,-80.201
                       Midtown  25.807,-80.193
                      Overtown  25.787,-80.201
                     Park West  25.785,-80.193
                     The Roads  25.756,-80.207
             

### Get only 'cool' neighborhoods

In [275]:
df = df.loc[df['Neighborhood'].isin(['Brickell','Coconut Grove','Design District','Downtown',
                                     'Little Havana','Midtown','Upper Eastside','Wynwood'])]
print(df.to_string(index = False))

    Neighborhood     Coordinates
        Brickell  25.758,-80.193
   Coconut Grove  25.712,-80.257
 Design District  25.813,-80.193
        Downtown  25.774,-80.193
   Little Havana  25.773,-80.215
         Midtown  25.807,-80.193
  Upper Eastside  25.830,-80.183
         Wynwood  25.804,-80.199


### Break out Latitude and Longitude from Coordinates

In [276]:
df['Latitude'] = df['Coordinates'].str[0:6].astype(float)
df['Longitude'] = df['Coordinates'].str[7:14].astype(float)
del df['Coordinates']
print (df)

       Neighborhood  Latitude  Longitude
3          Brickell    25.758    -80.193
5     Coconut Grove    25.712    -80.257
7   Design District    25.813    -80.193
8          Downtown    25.774    -80.193
15    Little Havana    25.773    -80.215
17          Midtown    25.807    -80.193
21   Upper Eastside    25.830    -80.183
25          Wynwood    25.804    -80.199


### Use geopy library to get the latitude and longitude values for Miami Beach and Coral Gables, FL

In [277]:
geolocator = Nominatim(user_agent="to_explorer")
addresses = ['Miami Beach', 'Coral Gables']

for address in addresses:
    location = geolocator.geocode(address + ', FL')
    latitude = location.latitude
    longitude = location.longitude
    df = df.append({'Neighborhood' : address, 'Latitude' : latitude, 'Longitude' : longitude } , ignore_index=True)


### Add Miami Beach and Coral Gables neighborhoods which are missing in the dataframe

In [278]:
df = df.sort_values(by=['Neighborhood'])

# Change index to start with one
df.index = np.arange(1, len(df) + 1)
 
print(df)

       Neighborhood  Latitude  Longitude
1          Brickell  25.75800 -80.193000
2     Coconut Grove  25.71200 -80.257000
3      Coral Gables  25.72149 -80.268384
4   Design District  25.81300 -80.193000
5          Downtown  25.77400 -80.193000
6     Little Havana  25.77300 -80.215000
7       Miami Beach  25.79292 -80.135301
8           Midtown  25.80700 -80.193000
9    Upper Eastside  25.83000 -80.183000
10          Wynwood  25.80400 -80.199000


### Use geopy library to get the latitude and longitude values for Miami, FL

In [279]:
address = 'Miami, FL'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Miami, FL are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Miami, FL are 25.7742658, -80.1936589.


In [280]:
# create map of Miami using latitude and longitude values
map_miami = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for index, lat, lng in zip(df.index, df['Latitude'], df['Longitude']):
    folium.Marker(location=[lat, lng],
                  icon=folium.DivIcon(html=f"""<div style="font-weight: bold; font-size: 25px; color: blue;">{index}</div>""")
                 ).add_to(map_miami)

display(map_miami) 

In [292]:
CLIENT_ID = '' #Foursquare ID
CLIENT_SECRET = '' #Foursquare Secret
VERSION = '20180605' #Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Get the venues for the neighborhoods.

In [282]:
def getVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [283]:
miami_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Brickell
Coconut Grove
Coral Gables
Design District
Downtown
Little Havana
Miami Beach
Midtown
Upper Eastside
Wynwood


In [284]:
print(miami_venues.shape)
miami_venues.head()

(248, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Brickell,25.758,-80.193,PM Buenos Aires Fish & Steak•House,25.758777,-80.193550,Argentinian Restaurant
1,Brickell,25.758,-80.193,"Edge, Steak & Bar",25.758960,-80.191692,Steakhouse
2,Brickell,25.758,-80.193,Obba Sushi & More,25.756834,-80.191018,Japanese Restaurant
3,Brickell,25.758,-80.193,Novecento,25.760267,-80.192490,Argentinian Restaurant
4,Brickell,25.758,-80.193,Valentia Miami,25.758641,-80.193133,Restaurant


In [285]:
miami_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Brickell,42,42,42,42,42,42
Coconut Grove,2,2,2,2,2,2
Coral Gables,1,1,1,1,1,1
Design District,21,21,21,21,21,21
Downtown,40,40,40,40,40,40
Little Havana,13,13,13,13,13,13
Miami Beach,31,31,31,31,31,31
Midtown,44,44,44,44,44,44
Upper Eastside,4,4,4,4,4,4


In [286]:
print('There are {} uniques categories.'.format(len(miami_venues['Venue Category'].unique())))

There are 51 uniques categories.


In [287]:
# one hot encoding
miami_onehot = pd.get_dummies(miami_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
miami_onehot['Neighborhood'] = miami_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [miami_onehot.columns[-1]] + list(miami_onehot.columns[:-1])
miami_onehot = miami_onehot[fixed_columns]

miami_onehot.head()

,Neighborhood,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Deli / Bodega,Diner,Eastern European Restaurant,Empanada Restaurant,Fast Food Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kosher Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Peruvian Restaurant,Pizza Place,Poke Place,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,South American Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant
0,Brickell,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Brickell,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Brickell,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Brickell,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Brickell,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [288]:
miami_onehot.shape

(248, 52)

In [289]:
miami_grouped = miami_onehot.groupby('Neighborhood').mean().reset_index()
miami_grouped

,Neighborhood,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Deli / Bodega,Diner,Eastern European Restaurant,Empanada Restaurant,Fast Food Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Kosher Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Peruvian Restaurant,Pizza Place,Poke Place,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,South American Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant
0,Brickell,0.095238,0.000000,0.071429,0.023810,0.000,0.000000,0.000,0.000000,0.000000,0.023810,0.095238,0.000000,0.000000,0.000000,0.023810,0.00,0.000,0.00,0.000000,0.047619,0.000000,0.000000,0.023810,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.095238,0.095238,0.00,0.023810,0.023810,0.023810,0.02381,0.000000,0.023810,0.00,0.095238,0.02381,0.047619,0.047619,0.000000,0.023810,0.023810,0.000000,0.000000,0.000000,0.000000,0.02381
1,Coconut Grove,0.500000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,Coral Gables,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,Design District,0.095238,0.047619,0.000000,0.047619,0.000,0.047619,0.000,0.000000,0.000000,0.000000,0.142857,0.000000,0.047619,0.000000,0.000000,0.00,0.000,0.00,0.047619,0.000000,0.047619,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.047619,0.000,0.095238,0.095238,0.00,0.000000,0.047619,0.000000,0.00000,0.000000,0.142857,0.00,0.000000,0.00000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,Downtown,0.050000,0.000000,0.000000,0.000000,0.025,0.000000,0.025,0.050000,0.025000,0.000000,0.075000,0.000000,0.025000,0.000000,0.000000,0.00,0.025,0.00,0.000000,0.000000,0.000000,0.000000,0.025000,0.0,0.000000,0.025,0.075000,0.000000,0.025,0.125000,0.000000,0.00,0.025000,0.000000,0.000000,0.05000,0.100000,0.050000,0.00,0.025000,0.02500,0.100000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.00000
5,Little Havana,0.000000,0.000000,0.000000,0.000000,0.000,0.153846,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.076923,0.00,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.384615,0.000000,0.076923,0.00000,0.000000,0.000000,0.00,0.076923,0.00000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.00000
6,Miami Beach,0.032258,0.000000,0.000000,0.000000,0.000,0.032258,0.000,0.000000,0.032258,0.000000,0.064516,0.000000,0.032258,0.129032,0.032258,0.00,0.000,0.00,0.000000,0.000000,0.064516,0.000000,0.032258,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.129032,0.000000,0.00,0.000000,0.032258,0.032258,0.00000,0.000000,0.161290,0.00,0.032258,0.00000,0.000000,0.000000,0.000000,0.000000,0.032258,0.032258,0.000000,0.064516,0.032258,0.00000
7,Midtown,0.022727,0.022727,0.022727,0.022727,0.000,0.045455,0.000,0.022727,0.022727

### Get the percentage of restaurants in each neighborhood

In [290]:
num_top_venues = 30

for neighborhood in miami_grouped['Neighborhood']:
    print(neighborhood)
    temp = miami_grouped[miami_grouped['Neighborhood'] == neighborhood].T.reset_index()
    temp.columns = ['Restaurant','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    temp.drop(temp[temp['Frequency'] == 0.00].index, inplace = True) 
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Brickell
                   Restaurant  Frequency
0         American Restaurant       0.10
1                        Café       0.10
2          Italian Restaurant       0.10
3         Japanese Restaurant       0.10
4                  Restaurant       0.10
5      Argentinian Restaurant       0.07
6          Seafood Restaurant       0.05
7                        Food       0.05
8              Sandwich Place       0.05
9                 Pizza Place       0.02
10                 Steakhouse       0.02
11         Spanish Restaurant       0.02
12                Salad Place       0.02
13   Mediterranean Restaurant       0.02
14    New American Restaurant       0.02
15         Mexican Restaurant       0.02
16  Latin American Restaurant       0.02
17          French Restaurant       0.02
18              Deli / Bodega       0.02
19               Burger Joint       0.02
20           Asian Restaurant       0.02
21      Venezuelan Restaurant       0.02


Coconut Grove
            Restaurant  Frequenc

In [291]:
for col in miami_grouped.columns: 
    print(col) 

Neighborhood
American Restaurant
Arepa Restaurant
Argentinian Restaurant
Asian Restaurant
Bagel Shop
Bakery
Bistro
Brazilian Restaurant
Breakfast Spot
Burger Joint
Café
Caribbean Restaurant
Chinese Restaurant
Cuban Restaurant
Deli / Bodega
Diner
Eastern European Restaurant
Empanada Restaurant
Fast Food Restaurant
Food
Food Court
Food Truck
French Restaurant
Fried Chicken Joint
Gastropub
Gluten-free Restaurant
Greek Restaurant
Indian Restaurant
Indonesian Restaurant
Italian Restaurant
Japanese Restaurant
Kosher Restaurant
Latin American Restaurant
Mediterranean Restaurant
Mexican Restaurant
New American Restaurant
Peruvian Restaurant
Pizza Place
Poke Place
Restaurant
Salad Place
Sandwich Place
Seafood Restaurant
South American Restaurant
Spanish Restaurant
Steakhouse
Sushi Restaurant
Taco Place
Tapas Restaurant
Vegetarian / Vegan Restaurant
Venezuelan Restaurant
